In [ ]:
import vectorbt as vbt
import pandas_ta as ta 
import pandas as pd 
import warnings
import numpy as np
import plotly.io as pio
pio.renderers.default = 'notebook'


df = vbt.YFData.download('BTC-USD')


warnings.simplefilter(action='ignore',category=FutureWarning)
pd.set_option('display.max_rows',10000)

In [ ]:
origin_df = pd.merge(pd.merge(pd.merge(pd.merge(df.get('Open'),df.get('Close'), on = 'Date')  ,df.get('High'), on = 'Date') ,df.get('Low'), on = 'Date')  ,df.get('Volume'), on = 'Date')

In [ ]:
origin_df = origin_df.reset_index().rename(columns={'Date':'Datetime'})
origin_df['Datetime'] = origin_df['Datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
origin_df.info()

In [ ]:
df_result = origin_df.set_index('Datetime')
df_result

In [ ]:
df_result.ta.cores=4

RSI

In [ ]:
rsi = ta.rsi(df_result['Close'],length=14)
df_result['rsi'] = rsi

In [ ]:
entries = df_result.apply(lambda x : (x['rsi']<30),axis=1)
print(entries.value_counts())

In [ ]:
exits = df_result.apply(lambda x : (x['rsi']>70),axis=1)
print(exits.value_counts())

In [ ]:
pf = vbt.Portfolio.from_signals(df_result['Close'],entries=entries,exits=exits,init_cash=1000,fees=0.01)

print(pf.stats())

In [ ]:
print(pf.orders.records_readable)

In [ ]:
pf.orders.plot(width = 1000)